In [4]:
import pandas as pd
import numpy as np

In [5]:
def createDataSet():
    row_data = {'1社会重要性':[3,2,1,1,1,1,3,1,1,1,2,2,1,1,2,3,1,1,1,1],
                '2用电可靠性':[3,3,2,3,2,3,2,3,2,2,3,2,3,3,2,3,1,1,2,3],
                '3用电量':[3,2,3,1,2,2,3,3,3,3,3,2,3,1,2,3,2,2,1,3],
                '4电价高低':[1,1,1,3,1,1,2,3,3,1,1,1,1,1,1,2,3,1,2,1],
                '5客户信用':[3,2,3,1,2,2,3,3,3,3,3,3,1,3,1,1,3,3,3,2],
                '6客户分类':['A','A','A','B','C','B','A','A','A','A','A','B','B','C','C','A','C','C','B','C']}
    dataSet = pd.DataFrame(row_data)
    return dataSet

In [6]:
dataSet=createDataSet()
dataSet

,1社会重要性,2用电可靠性,3用电量,4电价高低,5客户信用,6客户分类
0,3,3,3,1,3,A
1,2,3,2,1,2,A
2,1,2,3,1,3,A
3,1,3,1,3,1,B
4,1,2,2,1,2,C
5,1,3,2,1,2,B
6,3,2,3,2,3,A
7,1,3,3,3,3,A
8,1,2,3,3,3,A
9,1,2,3,1,3,A


In [11]:
def calEnt(dataSet):
    n = dataSet.shape[0];                     #得到数据集行数
    iset = dataSet.iloc[:,-1].value_counts()  #所有标签得类别
    p = iset / n                              #每一标签所占比重
    ent = ( - p * np.log2(p)).sum()           #计算信息熵
    return ent

In [12]:
calEnt(dataSet)

1.5394910703001345

In [44]:
#选择最优的列进行切分
def bestSplit(dataSet):
    baseEnt = calEnt(dataSet)                 #计算原始信息熵
    bestGain = 0                              #初始化信息增益
    axis = -1                                 #初始化最佳分割，标签列
    for i in range(dataSet.shape[-1] - 1):
        levels = dataSet.iloc[:,i].value_counts().index
        ents = 0
        for j in levels:
            childSet = dataSet[dataSet.iloc[:,i] == j]
            ent = calEnt(childSet)
            ents += (childSet.shape[0]/dataSet.shape[0])*ent
        infoGain = baseEnt - ents
        print(infoGain)
        if(infoGain > bestGain):
            bestGain = infoGain
            axis = i
    print(dataSet.columns[axis])
    return axis

In [45]:
bestSplit(dataSet)

0.21468727723026526
0.19675342168646726
0.45820842502811865
0.09516930095210552
0.10883216769577575
3用电量


2

In [37]:
def mySplit(dataSet,axis,value):
    col = dataSet.columns[axis]
    redataSet = dataSet.loc[dataSet[col]==value,:].drop(col,axis = 1)
    return redataSet

In [38]:
mySplit(dataSet,0,1)

,2用电可靠性,3用电量,4电价高低,5客户信用,6客户分类
2,2,3,1,3,A
3,3,1,3,1,B
4,2,2,1,2,C
5,3,2,1,2,B
7,3,3,3,3,A
8,2,3,3,3,A
9,2,3,1,3,A
12,3,3,1,1,B
13,3,1,1,3,C
16,1,2,3,3,C


In [46]:
def createTree(dataSet):
    featlist = list(dataSet.columns)
    classlist = dataSet.iloc[:,-1].value_counts()
    if classlist[0] == dataSet.shape[0] or dataSet.shape[1] == 1:
        return classlist.index[0]
    axis = bestSplit(dataSet)
    bestfeat = featlist[axis]
    myTree = {bestfeat:{}}
    del featlist[axis]                                                  #删除当前特征
    print(dataSet)
    valuelist = set(dataSet.iloc[:,axis])                               #提取最佳切分列所有属性值
    for value in valuelist:                                             #对每一个属性值递归建树
        myTree[bestfeat][value] = createTree(mySplit(dataSet,axis,value))
    return myTree

In [47]:
myTree = createTree(dataSet)

0.21468727723026526
0.19675342168646726
0.45820842502811865
0.09516930095210552
0.10883216769577575
3用电量
    1社会重要性  2用电可靠性  3用电量  4电价高低  5客户信用 6客户分类
0        3       3     3      1      3     A
1        2       3     2      1      2     A
2        1       2     3      1      3     A
3        1       3     1      3      1     B
4        1       2     2      1      2     C
5        1       3     2      1      2     B
6        3       2     3      2      3     A
7        1       3     3      3      3     A
8        1       2     3      3      3     A
9        1       2     3      1      3     A
10       2       3     3      1      3     A
11       2       2     2      1      3     B
12       1       3     3      1      1     B
13       1       3     1      1      3     C
14       2       2     2      1      1     C
15       3       3     3      2      1     A
16       1       1     2      3      3     C
17       1       1     2      1      3     C
18       1       2     1      2      3  

In [42]:
myTree

{'3用电量': {1: {'4电价高低': {1: 'C', 2: 'B', 3: 'B'}},
  2: {'2用电可靠性': {1: 'C',
    2: {'5客户信用': {1: 'C', 2: 'C', 3: 'B'}},
    3: {'1社会重要性': {1: 'B', 2: 'A'}}}},
  3: {'5客户信用': {1: {'1社会重要性': {1: 'B', 3: 'A'}}, 2: 'C', 3: 'A'}}}}

In [22]:
#导入相应的包
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
import graphviz

ModuleNotFoundError: No module named 'graphviz'